# Annotation data 
> selecting data for annotation.

In [1]:
# hide
from nbdev.showdoc import *

In [2]:
# export
%load_ext autoreload
%autoreload 2
import pandas as pd
from tqdm import tqdm_notebook
from whatsapp_analytics import translation
from utils import preprocess
import pickle
from os import path
import parmap
import numpy as np
from langdetect import DetectorFactory
DetectorFactory.seed = 0
from langdetect import detect
import re
import uuid

In [3]:
parent_path='../Data/New_Data_15-06-2020/'

In [4]:
def add_translation(df):
    translated_text_list=[]
    num_keywords =[] 
    language = []
    for index,row in tqdm_notebook(df.iterrows(),total=len(df)):
        temp = re.sub(r"http\S+", "", row["message_text"])
        lang=row['language']
        if(lang in ['en','unk']):
            translated_text_list.append(temp)
        else:
            translated_text=translation.translate(row["message_text"],lang,to_preprocess=False)
            translated_text_list.append(translated_text)    
    df["translated"]=translated_text_list
    return df



def add_keywords(df,type1='hindu'):
    filename='../Data/Important_lexicon/'+type1+'_keywords_complete.txt'
    keyword_text = open(filename, 'r')
    keywords = [line.strip('\n').lower() for line in keyword_text.readlines()]
    
    num_keywords =[] 
    for index,row in tqdm_notebook(df.iterrows(),total=len(df)):
        temp = row['tokenized']
        keywords_found=[x for x in temp if x in keywords]
        count=len(keywords_found)
        num_keywords.append(count)
    colname=type1+"_keyword_count"
    df[colname]=num_keywords
    return df

### Steps taken to further to collect the data 
1. save ids processed for annotation
2. add translate 
3. add keyword
4. Add uuids
5. save annotated data

In [7]:
# hide
if path.exists(parent_path+'data_processed_for_annotation.pkl'):
    processed_df = pd.read_pickle(parent_path+'data_processed_for_annotation.pkl')
else:
    processed_df=pd.DataFrame(columns=['message_text','translated','keyword_count','repeated','language','unique_ids'])

In [ ]:
#processed_df = pd.read_csv(parent_path+'Data_text_spam_removed_v02.csv')

In [8]:
group_names = pd.read_csv(parent_path+'group_party_affiliation.csv',sep=',')

In [9]:
total_group_annotation=pd.read_csv('../Data/New_Data_15-06-2020/group_annotation/all_annotation_data_round12.csv')

In [10]:
dict_group_type={}
for index,row in group_names.iterrows():
    dict_group_type[int(row['group_id'])]=row['party_name']
    
    
dict_group_type_new={}
for index,row in total_group_annotation.iterrows():
    dict_group_type_new[int(row['grpid'])]=row['annotated_punyajoy']

In [11]:
if path.exists(parent_path+'data_processed_for_annotation.pkl')==False:
    group_type = []
    for index,row in tqdm_notebook(processed_df.iterrows(),total=len(processed_df)):
        try:
            group_type.append(dict_group_type[row['group_id_anonymized']])
        except KeyError:
            try:
                group_type.append(dict_group_type_new[row['group_id_anonymized']])
            except KeyError:
                group_type.append('None')

    processed_df['group_type']=group_type

In [12]:
len(processed_df[processed_df['group_type']=='None']['group_id_anonymized'].unique())

592

In [22]:
#processed_df=processed_df.loc[:, ~processed_df.columns.str.match('Unnamed')]

In [13]:
if path.exists(parent_path+'data_processed_for_annotation.pkl')==False:
    processed_df=preprocess(processed_df)
    processed_df=add_keywords(processed_df,type1='hindu')
    processed_df=add_keywords(processed_df,type1='muslim')
    

In [14]:
#processed_df=add_keywords(processed_df,type1='muslim')

In [15]:
if path.exists(parent_path+'data_id_to_uuid.pkl'):
    with open(parent_path+'data_id_to_uuid.pkl', 'rb') as f:
        mapping = pickle.load(f)
else:
    mapping = {}

In [16]:
if path.exists(parent_path+'data_processed_for_annotation.pkl')==False:
    num_annotators_per_text=3
    list_of_unique_ids=[]
    for index,row in tqdm_notebook(processed_df.iterrows(),total=len(processed_df)):
        list_of_mapping=[]
        for i in range(num_annotators_per_text):
            id1=str(uuid.uuid4())
            list_of_mapping.append(id1)
            mapping[id1]=index+previous_len
        list_of_unique_ids.append(list_of_mapping)
    processed_df['unique_ids']=list_of_unique_ids

In [17]:
previous_len

NameError: name 'previous_len' is not defined

In [18]:
# hide 
## saving the whole results to be used later
if path.exists(parent_path+'data_processed_for_annotation.pkl')==False:
    with open(parent_path+'data_id_to_uuid.pkl', 'wb') as f:
        pickle.dump(mapping,f)
    processed_df.to_pickle(parent_path+'data_processed_for_annotation.pkl')

In [19]:
#processed_df.to_pickle(parent_path+'data_processed_for_annotation.pkl')

In [20]:
processed_df.head(5)

,group_id_anonymized,phone_num_anonymized,message_text,timestamp,media_type,media_url,forwarded,orig_index,language,only_urls,is_spam,group_type,tokenized,hindu_keyword_count,muslim_keyword_count,unique_ids,preds,pred_probab
0,4,4,కోడెల కుల ప్రసంగం :- వేల కోట్లు ఖర్చు పెట్టి...,1549564818000,None,None,1,3,te,0,0,YSRCP,"[కోడెల, కుల, ప్రసంగం, వేల, కోట్లు, ఖర్చు, పెట్...",0,0,"[d5c46b9b-529f-41d5-b13f-3ce7d87167af, 34ec1f7...",0,0.008496
1,5,5,Chirala mla paristiti enti frnds cheppandi,1549559412000,None,None,0,4,it,0,0,YSRCP,"[chirala, mla, paristiti, enti, frnds, cheppandi]",0,0,"[fae5c0c6-4803-4d4f-b18a-e5f18f16d66f, de1cdad...",0,0.053958
2,9,9,Dilli me aaj ole,1549559506000,None,None,0,10,et,0,0,BJP,"[dilli, me, aaj, ole]",0,0,"[b1cbc961-5c4d-450a-91d5-71a66d43494f, 2c7f282...",0,0.126118
3,9,9,Me gajendra dilli,1549559512000,None,None,0,11,sq,0,0,BJP,"[me, gajendra, dilli]",0,0,"[6de62602-6f1b-4f60-8d5c-238837f50657, 9adceea...",1,0.657664
4,13,12,*लघु सिंचाई* निःशुल्क बोरिंग योजना हेतु 55 कर...,1549559608000,None,None,1,16,hi,0,0,SP,"[लघु, सिंचाई, निःशुल्क, बोरिंग, योजना, हेतु, क...",0,0,"[f265737e-0dd2-4cd8-9f31-5220a94c8dba, c7d9c2a...",0,0.005101


### Selection function for dataset

In [21]:
# hide
def model_based_selection(annotated_data,unlabeled_data,model='lr'):
    ## curently implements random
    sample_df=unlabeled_data.sample(num_data_points)
    return sample_df

def select_data(unlabeled_data,ids_done,num_data_points=200,way_of_selection='random', num_keywords_hindu=0,num_keywords_muslim=3,parties_to_consider=['BJP','HINDU'],thresh=0.7):
    temp=unlabeled_data.drop(list(ids_done))
    if(parties_to_consider ==['all']):
        temp=temp[temp['pred_probab'] >= thresh]
    else:
        temp=temp[(temp['group_type'].isin(parties_to_consider)) & (temp['pred_probab'] >= thresh)]
    if(way_of_selection=='random'):
        sample_df=temp.sample(num_data_points)
    elif(way_of_selection=='keyword_based'):
        filtered_data=temp[(temp['hindu_keyword_count']>=num_keywords_hindu) & (temp['muslim_keyword_count']>=num_keywords_muslim)]
        print(len(filtered_data))
        sample_df=filtered_data.sample(num_data_points)
    return sample_df

In [22]:
if path.exists(parent_path+'annotated_ids.pkl'):
    with open(parent_path+'annotated_ids.pkl', 'rb') as f:
        ids_done = pickle.load(f)
else:
    ids_done = []

In [23]:
total_group_annotation['annotated_punyajoy'].unique()

array(['YSRCP', 'DOUBT', 'JANASENA', 'HINDU', 'OTHERS', 'SP', 'CASTE',
       'BJP', 'MUSLIM', 'AAP', 'TMC', 'RJD', 'CONGRESS', 'AIMIM', 'BSP',
       'O', 'NCP', 'Anti-BJP', 'NEWS', 'HATE', 'TRS', 'AntiBJP',
       'Abhinav Rajasthan Party', 'SFI', 'Jansatta Dal', 'AntiCONGRESS',
       'TDP', 'RLP', 'Hate'], dtype=object)

In [24]:
#sample_df=select_data(processed_df,ids_done,num_data_points=2000,way_of_selection='keyword_based',num_keywords_hindu=0,num_keywords_muslim=2,parties_to_consider=['BJP','HINDU','DOUBT','OTHER','HATE','Hate','CONGRESS','AntiBJP','AntiCONGRESS'], thresh=0.1)

In [25]:
sample_df=select_data(processed_df,ids_done,num_data_points=4000,way_of_selection='keyword_based',num_keywords_hindu=0,num_keywords_muslim=2,parties_to_consider=['BJP','HINDU','OTHER','HATE','Hate','DOUBT'], thresh=0.7)

6503


In [ ]:
del processed_df

In [26]:
from whatsapp_analytics.marking_similars import *
thresh_to_consider = 0.7

def remove_duplicates_within(sample_df):
    sample_df=add_signatures(sample_df,numHashes)
    id_done=0
    duplicate={}
    for index,row in tqdm_notebook(sample_df.iterrows(),total=len(sample_df)):
        id_done+=1
        try:
            temp_dupl=duplicate[index]
            continue
        except KeyError:
            temp=sample_df[id_done:]
            for index1,row1 in temp.iterrows():
                try:
                    temp_dupl=duplicate[index]
                    continue
                except KeyError:
                    signature1=row['signatures']
                    signature2=row1['signatures']
                    count=0
                    for k in range(0, numHashes):
                        count = count + (signature1[k] == signature2[k])
                    # add to tuple similar if greater than thresh    
                    if((count/numHashes)>thresh_to_consider):
                        duplicate[index1]=1
    
    sample_df=sample_df.drop(list(duplicate.keys()))
    return sample_df


def remove_duplicates_ext(sample_df,total_annotation):
    df_total=pd.concat([sample_df,total_annotation])
    df_total=add_signatures(df_total,numHashes)
    sample_df=df_total[0:len(sample_df)]
    total_annotation=df_total[len(sample_df):]
    duplicate={}
    for index,row in tqdm_notebook(sample_df.iterrows(),total=len(sample_df)):
        for index1,row1 in total_annotation.iterrows():
            signature1=row['signatures']
            signature2=row1['signatures']
            count=0
            for k in range(0, numHashes):
                count = count + (signature1[k] == signature2[k])
            # add to tuple similar if greater than thresh    
            if((count/numHashes)>thresh_to_consider):
                duplicate[index]=1
    sample_df=sample_df.drop(list(duplicate.keys()))
    return sample_df
 

In [27]:
from glob import glob
import pandas as pd
from tqdm import tqdm_notebook

annotation_data='../Data/Annotations_to_send/*'
filenames=glob(annotation_data)

list_df=[]
for file in filenames:
    if(file[-3:]=='pkl'):
        df=pd.read_pickle(file)
    if(file[-3:]=='csv'):
        df=pd.read_csv(file)
    list_df.append(df)

    
df_total=pd.concat(list_df)


/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [28]:
sample_df_red=remove_duplicates_within(sample_df)
sample_df_final=remove_duplicates_ext(sample_df_red,df_total)

100%|██████████| 4000/4000 [00:08<00:00, 496.17it/s]


1710566


/home/punyajoy/.conda/envs/punyajoy_gpu/lib/python3.7/site-packages/ipykernel_launcher.py:34: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  0%|          | 30/7331 [00:00<00:25, 288.43it/s]

100%|██████████| 7331/7331 [00:15<00:00, 460.78it/s]


3328616


In [30]:
sample_df_final=sample_df_final[0:200]

227

In [31]:
n_process = 10
size=100
list_df=[]
for i in tqdm_notebook(range(0,len(sample_df_final),size)):
    df_split = np.array_split(sample_df_final[i:i+size], n_process)
    df_list = parmap.map(add_translation, df_split,pm_processes=n_process)    
    new_df_processed = pd.concat(df_list,ignore_index=True)
    list_df.append(new_df_processed)

sample_df_final= pd.concat(list_df,ignore_index=True)
    

#sample_df_final=add_translation(sample_df_final)

In [32]:
temp_to_check=sample_df_final[['message_text','translated','group_type','orig_index','muslim_keyword_count','hindu_keyword_count']][20:40]

In [33]:
temp_to_check

,message_text,translated,group_type,orig_index,muslim_keyword_count,hindu_keyword_count
20,मै हूँ ऐसा हिंदू..|| जो मुल्लो की बहन चोदने म...,I am such a Hindu .. || The one who is Mulloy'...,OTHER,3806603.0,13.0,8.0
21,#कोंग्रेस और उसके सहयोगी दल #पाकिस्तान_परस्त क...,Why are #Congress and its allies #Pakistan_Par...,BJP,3879566.0,3.0,0.0
22,*🚩🌞धर्म संस्थापनार्थाय🌞🚩* *⛳🇮🇳 सनातन धर्म की ...,* 🚩 🌞 Religion for the 🚩 🌞 🇮 🇳 ⛳ Establ...,OTHER,2448924.0,2.0,5.0
23,BEFORE YOU SURRENDER YOUR GUNS TO THE POLICE O...,BEFORE YOU SURRENDER YOUR GUNS TO THE POLICE O...,OTHER,4718214.0,2.0,0.0
24,कश्मीर का इस्लामीकरण: हिन्दुओं को सीख डॉ विवे...,Islamization of Kashmir: Learning to Hindus Dr...,OTHER,5144608.0,9.0,12.0
25,*😡😡😡 ये है ईसाई मिशनरियों का धर्मान्तरण का और ...,* 😡 😡 😡 है This is the conversion of Christ...,BJP,5345582.0,3.0,6.0
26,सम्राट अशोक का अहिंसक बौद्ध बनना हिन्दुत्व की ...,Emperor Ashoka's becoming a non-violent Buddhi...,BJP,2953482.0,3.0,8.0
27,हमारे गौरवशाली इतिहास का एक प्रेरणादायक पृष्ठ ...,"An inspirational page of our glorious history,...",BJP,5223531.0,3.0,6.0
28,पत्नी बार बार मां पर इल्जाम लगाए जा रही थी.......,The wife was repeatedly blaming the mother …… ...,BJP,4795592.0,2.0,0.0
29,ख़ौफ़ज़दा महबूबा : जो बोया था वो काटने का समय ...,Khoufzada Mehbooba: Do not commit the crime of...,BJP,1537216.0,7.0,4.0


In [34]:
temp_to_check['Annotation']=['None']*len(temp_to_check)
temp_to_check.to_csv('pilot-checking_fear_speech_round10.csv')

### Only proceed if data is approved

In [35]:
ids_done += list(sample_df_final.index)

In [36]:
with open(parent_path+'annotated_ids.pkl', 'wb') as f:
        pickle.dump(ids_done,f)
    

In [37]:
if path.exists(parent_path+'round_info.pkl'):
    with open(parent_path+'round_info.pkl', 'rb') as f:
        round_info = pickle.load(f)
else:
    round_info = 2

In [38]:
round_info

10

In [39]:
sample_df_final=sample_df_final[['forwarded', 'group_id_anonymized', 'group_type',
       'hindu_keyword_count', 'language', 'message_text', 'muslim_keyword_count', 'orig_index', 'phone_num_anonymized', 'timestamp', 'translated', 'unique_ids']]

In [40]:
round_info+=1

In [41]:
annotation_data='../Data/Annotations_to_send/'
sample_df_final.to_pickle(annotation_data+'annotation_'+str(round_info)+'.pkl')

In [42]:
with open(parent_path+'round_info.pkl', 'wb') as f:
    pickle.dump(round_info,f)
    

In [43]:
round_info

11

In [ ]:
pd.set_option('display.max_colwidth', 1000)

sample_df_final[['message_text']].sample(10)